In [1]:
using Revise

using RigidBodyDynamics
using RigidBodyDynamics: Bounds
using LCPSim
# using Ipopt
using ConditionalJuMP
using Gurobi
using RigidBodyTreeInspector
using DrakeVisualizer

INFO: Recompiling stale cache file /Users/blandry/.julia/lib/v0.6/LCPSim.ji for module LCPSim.


HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/8c8a043c2f00369f3e7ae550277d2a54e59a34ad-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/8c8a043c2f00369f3e7ae550277d2a54e59a34ad-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [2]:
urdf = joinpath("..","urdf", "brick.urdf")
mechanism = parse_urdf(Float64, urdf)

body = findbody(mechanism, "brick")
basejoint = joint_to_parent(body, mechanism)
floatingjoint = Joint(basejoint.name, frame_before(basejoint), frame_after(basejoint), QuaternionFloating{Float64}())
replace_joint!(mechanism, basejoint, floatingjoint)

position_bounds(floatingjoint) .= Bounds(-100, 100)
velocity_bounds(floatingjoint) .= Bounds(-100, 100);

In [25]:
x0 = MechanismState(mechanism)
set_configuration!(x0, [1., 0., 0., 0., 0., 0., 2.])
set_velocity!(x0, [0., 1., 0., 0., 1. , 0.])
env = LCPSim.parse_contacts(mechanism, urdf);
controller = LCPSim.passive_controller()
Δt = 0.01
N = 100

100

In [11]:
q = [1., 0., 0., 0., 0., 0., 2.]
v = [0., 0., 0., 0., 0. , 0.]
u = [0., 0., 0., 0., 0. , 0.]
eval_g, eval_jac_g = update_constraints_implicit(env,mechanism,Δt,q,v,u)

(LCPSim.eval_g, LCPSim.eval_jac_g)

In [5]:
x = rand(length(q)+length(v))
g = zeros(length(x)+length(env.contacts))
eval_g(x, g)
values = zeros(length(g)*length(x))
eval_jac_g(x, :Eval, [], [], values)

221-element Array{Float64,1}:
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 1.0        
 0.0        
 0.0        
 0.0        
 2.00195e-12
 0.0        
 ⋮          
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        

In [26]:
results = LCPSim.simulate(x0, controller, env, Δt, N)

This is Ipopt version 3.12.8, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      169
Number of nonzeros in inequality constraint Jacobian.:       52
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       13
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       13
                     variables with only upper bounds:        0
Total number of equality constraints.................:       13
Total number of inequality constraints...............:        4
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        4
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

13×101 Array{Float64,2}:
 1.0   1.0           0.999975      0.999925     …   0.881203      0.878829   
 0.0   3.50325e-46  -1.85701e-21  -8.50419e-21     -2.45474e-16  -2.51313e-16
 0.0   0.005         0.00999994    0.0149997        0.475349      0.479749   
 0.0   0.0           3.55197e-22   7.08892e-22     -2.07925e-16  -2.14181e-16
 0.0   4.48416e-46  -2.45244e-10  -7.35742e-10      0.00092323    0.000961736
 0.0   0.01          0.02          0.03         …   0.99          1.0        
 2.0   1.99902       1.99706       1.99411         -2.86389      -2.96223    
 0.0   0.0          -3.71752e-19  -1.32974e-18     -7.50975e-16  -7.32984e-16
 1.0   1.0           1.0           1.0              1.0           1.0        
 0.0   0.0           6.91815e-20   5.9302e-20      -1.51112e-15  -1.53837e-15
 0.0   4.48416e-44   0.00196198    0.00588577   …   8.08432       8.22055    
 1.0   1.0           1.0           1.0              1.0           1.0        
 0.0  -0.0981       -0.196195     -0.29

In [19]:
qs = [configuration(r.state) for r in results]
ts = cumsum([r.Δt for r in results]);
for q in qs
    println(q)
end
# println(qs)
# plot([q[7] for q in qs])

LoadError: [91mtype Float64 has no field state[39m

In [20]:
results

13×11 Array{Float64,2}:
 1.0   1.0       1.0       1.0       1.0      …   1.0       1.0       1.0    
 0.0   0.0       0.0       0.0       0.0          0.0       0.0       0.0    
 0.0   0.0       0.0       0.0       0.0          0.0       0.0       0.0    
 0.0   0.0       0.0       0.0       0.0          0.0       0.0       0.0    
 0.0   0.0       0.0       0.0       0.0          0.0       0.0       0.0    
 0.0   0.0       0.0       0.0       0.0      …   0.0       0.0       0.0    
 2.0   1.99902   1.99706   1.99411   1.99019      1.96468   1.95586   1.94605
 0.0   0.0       0.0       0.0       0.0          0.0       0.0       0.0    
 0.0   0.0       0.0       0.0       0.0          0.0       0.0       0.0    
 0.0   0.0       0.0       0.0       0.0          0.0       0.0       0.0    
 0.0   0.0       0.0       0.0       0.0      …   0.0       0.0       0.0    
 0.0   0.0       0.0       0.0       0.0          0.0       0.0       0.0    
 0.0  -0.0981   -0.1962   -0.2943   -0.3

In [28]:
qs = [results[1:num_positions(x0),i] for i in 1:N]
ts = cumsum([Δt for i in 1:N]);

In [28]:
qs

10-element Array{Array{Float64,1},1}:
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0]    
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.99902]
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.99706]
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.99411]
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.99019]
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.98529]
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.9794] 
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.97253]
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.96468]
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.95586]

In [29]:
DrakeVisualizer.any_open_windows() || (DrakeVisualizer.new_window(); sleep(1));
geometries = visual_elements(mechanism, URDFVisuals(urdf))
vis = Visualizer(mechanism, geometries);

In [33]:
RigidBodyTreeInspector.animate(vis, mechanism, ts, qs; realtimerate = .5);

In [3]:
q0 = [1., 0., 0., 0., 0., 0., 2.]
v0 = [0., 1., 0., 0., 1. , 0.]
x0 = MechanismState(mechanism)
set_configuration!(x0, q0)
set_velocity!(x0, v0)
env = LCPSim.parse_contacts(mechanism, urdf);
controller = LCPSim.passive_controller()
Δt = 0.01
qnext = [1., 0., 0., 0., 0., 0., 2.]
vnext = [0., 1., 0., 0., 1. , 0.]
τ_contact_wrenches(env,mechanism,Δt,x0,qnext,vnext)

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] max(::Float64, ::Array{Float64,1}) at ./deprecated.jl:57
 [3] broadcast_t(::Function, ::Type{Any}, ::Tuple{Base.OneTo{Int64}}, ::CartesianRange{CartesianIndex{1}}, ::Array{Any,1}, ::Array{Array{Float64,1},1}) at ./broadcast.jl:258
 [4] broadcast_c at ./broadcast.jl:321 [inlined]
 [5] broadcast(::Function, ::Array{Any,1}, ::Array{Array{Float64,1},1}) at ./broadcast.jl:455
 [6] augmented_lagrangian_newton!(::Array{Float64,1}, ::LCPSim.#f#25, ::LCPSim.#h#26, ::LCPSim.#g#27{Int64,Array{Int64,1},Array{Int64,1},Array{Int64,1}}, ::Int64, ::Int64, ::Int64, ::Float64, ::Float64) at /Users/blandry/.julia/v0.6/LCPSim/src/bilevel.jl:21
 [7] τ_contact_wrenches(::LCPSim.Environment{Float64}, ::RigidBodyDynamics.Mechanism{Float64}, ::Float64, ::RigidBodyDynamics.MechanismState{Float64,Float64,Float64,TypeSortedCollections.TypeSortedCollection{Tuple{Array{RigidBodyDynamics.Joint{Float64,RigidBodyDynamics.Fixed{Float64}},1},Array{R

LoadError: [91mDimensionMismatch("Cannot multiply two vectors")[39m

In [37]:
n = 5
x = zeros(n)
f = x -> x'*x
h = x -> [0.]
g = x -> 5. - x
num_h = length(h(x))
num_g = length(g(x))
N = 5
α = 1.
I = 1e-12
augmented_lagrangian_newton!(x,f,h,g,num_h,num_g,N,α,I)

[4.91176, 4.91176, 4.91176, 4.91176, 4.91176]
[4.99913, 4.99913, 4.99913, 4.99913, 4.99913]
[5.0, 5.0, 5.0, 5.0, 5.0]
[5.0, 5.0, 5.0, 5.0, 5.0]
[5.0, 5.0, 5.0, 5.0, 5.0]


In [18]:
λ = ones(num_h)
μ = ones(num_g)
c = .01
L = (x,λ,μ,c) -> (f(x) + dot(λ,h(x)) + .5*c*dot(h(x),h(x)) + 1./(2.*c)*sum(max.([0.],μ.+c*g(x)).^2 - μ.^2))[1]
∇xL = (x,λ,μ,c) -> ForwardDiff.gradient(x̃ -> L(x̃,λ,μ,c),x)
HxL = (x,λ,μ,c) -> ForwardDiff.jacobian(x̃ -> ∇xL(x̃,λ,μ,c),x)
HxL(x,λ,μ,c)

5×5 Array{Float64,2}:
 2.01  0.0   0.0   0.0   0.0 
 0.0   2.01  0.0   0.0   0.0 
 0.0   0.0   2.01  0.0   0.0 
 0.0   0.0   0.0   2.01  0.0 
 0.0   0.0   0.0   0.0   2.01